# HEL-21 Phase 4: FastAPI Integration & Entity Linking

## 🎯 Phase 4 目标
1. ✅ 测试Phase 3训练的模型
2. 🔗 实现Entity Linking（模糊匹配）
3. 🚀 开发FastAPI endpoints
4. 🧪 集成测试

**注意**: 所有代码在 `.py` 文件中，此notebook仅用于运行和测试

---

## 📦 Step 1: 环境设置

## ⚠️ 重要：首次运行指南

### 第一次使用此notebook时：

1. **运行Cell 3**: 挂载Google Drive
2. **运行Cell 4**: 切换到项目目录
3. **运行Cell 5**: 修复依赖冲突（这会安装兼容的包版本）
4. **⚠️ 必须重启runtime**: `Runtime` → `Restart runtime`
5. **重启后运行Cell 6**: 验证环境
6. **继续后续cells**

### 已经完成setup的用户：
- 跳过Cell 5（依赖安装）
- 直接从Cell 6（验证）或Cell 7（测试模型）开始

---

In [1]:
# 切换到项目目录
%cd /content/drive/MyDrive/Helixgraph

# 验证目录结构
!pwd
!ls -la nlp/

/content/drive/MyDrive/Helixgraph
/content/drive/MyDrive/Helixgraph
total 103
drwx------ 2 root root  4096 Nov 21 16:44 configs
-rw------- 1 root root  8638 Nov 21 17:00 EDUCATIONAL_SUMMARY.md
-rw------- 1 root root  5569 Nov 23 19:56 entity_extraction.py
-rw------- 1 root root  6357 Nov 23 19:55 entity_linking.py
drwx------ 2 root root  4096 Nov 21 16:42 evaluation
-rw------- 1 root root 53064 Nov 22 22:15 HEL21_NER_Training.ipynb
drwx------ 2 root root  4096 Nov 23 19:10 models
-rw------- 1 root root  6196 Nov 21 16:43 README.md
drwx------ 2 root root  4096 Nov 23 19:55 scripts
drwx------ 2 root root  4096 Nov 21 16:34 task_descriptions
drwx------ 2 root root  4096 Nov 21 18:42 training_data


In [ ]:
# 🔧 最终修复方案 - 强制重新安装并降级thinc
import subprocess
import sys

print("=" * 80)
print("🔥 Final Fix - Force reinstall with compatible thinc")
print("=" * 80)

# Step 1: 完全移除spacy相关包
print("\n📦 Step 1: Completely removing spacy ecosystem...")
packages = [
    "spacy", "spacy-transformers", "spacy-legacy", "spacy-loggers", 
    "thinc", "catalogue", "srsly", "wasabi", "confection",
    "pydantic", "pydantic-core", "fastapi"
]
for pkg in packages:
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], 
                   capture_output=True)
print("   ✅ Removed all packages")

# Step 2: 清除缓存
print("\n🗑️  Step 2: Clearing ALL caches...")
subprocess.run([sys.executable, "-m", "pip", "cache", "purge"], capture_output=True)
subprocess.run(["rm", "-rf", "/root/.cache/pip"], capture_output=True)
print("   ✅ Caches cleared")

# Step 3: 使用force-reinstall安装
print("\n📥 Step 3: Force reinstalling with specific versions...")
install_sequence = [
    # 基础依赖
    ["pip", "install", "--force-reinstall", "--no-cache-dir", "pydantic==2.6.0", "pydantic-core==2.16.1"],
    
    # spaCy依赖（指定兼容版本）
    ["pip", "install", "--force-reinstall", "--no-cache-dir", 
     "catalogue==2.0.10", "srsly==2.4.8", "wasabi==1.1.2", "confection==0.1.4"],
    
    # thinc（使用稍早版本，可能更稳定）
    ["pip", "install", "--force-reinstall", "--no-cache-dir", "thinc==8.2.3"],
    
    # spaCy核心和扩展
    ["pip", "install", "--force-reinstall", "--no-cache-dir", 
     "spacy-legacy==3.0.12", "spacy-loggers==1.0.5"],
    ["pip", "install", "--force-reinstall", "--no-cache-dir", "spacy==3.7.4"],
    ["pip", "install", "--force-reinstall", "--no-cache-dir", "spacy-transformers==1.3.5"],
    
    # FastAPI
    ["pip", "install", "--no-cache-dir", "fastapi==0.109.0", "uvicorn[standard]==0.27.0"],
    
    # 其他工具
    ["pip", "install", "--no-cache-dir", 
     "fuzzywuzzy", "python-Levenshtein", "nest-asyncio", "pytest", "httpx"]
]

for i, cmd in enumerate(install_sequence, 1):
    print(f"\n   [{i}/{len(install_sequence)}] {' '.join(cmd[2:5])}...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"      ✅ Success")
    else:
        print(f"      ⚠️  Warning (may be OK)")

# Step 4: 验证
print("\n" + "=" * 80)
print("🔍 Quick verification...")
print("=" * 80)

result = subprocess.run([sys.executable, "-c", 
    "import spacy; import pydantic; print(f'spaCy: {spacy.__version__}'); print(f'pydantic: {pydantic.__version__}')"],
    capture_output=True, text=True)

if result.returncode == 0:
    print(result.stdout)
    print("\n✅ Basic imports work!")
else:
    print(result.stderr[:300])

print("\n" + "=" * 80)
print("⚠️  MUST RESTART RUNTIME NOW!")
print("=" * 80)
print("\nRuntime → Restart runtime")
print("\nIf this still fails after restart, we need to use Python 3.10")
print("=" * 80)

/content/drive/MyDrive/Helixgraph
/content/drive/MyDrive/Helixgraph


In [4]:
# ✅ 验证环境 (运行此cell前请先重启runtime)
print("=" * 80)
print("🔍 Verifying installation...")
print("=" * 80)

# 先检查安装的版本
print("\n📦 Checking installed versions:")
import subprocess
result = subprocess.run(["pip", "list"], capture_output=True, text=True)
for line in result.stdout.split('\n'):
    if any(pkg in line.lower() for pkg in ['spacy', 'pydantic', 'fastapi', 'thinc']):
        print(f"   {line}")

print("\n" + "=" * 80)
print("🧪 Testing imports...")
print("=" * 80)

try:
    import spacy
    import pydantic
    import fastapi
    from fuzzywuzzy import fuzz
    
    print(f"\n✅ spaCy version: {spacy.__version__}")
    print(f"✅ pydantic version: {pydantic.__version__}")
    print(f"✅ FastAPI version: {fastapi.__version__}")
    print(f"✅ fuzzywuzzy imported successfully")
    
    # Try to actually use spacy (this is where the error happens)
    print("\n🔬 Testing spaCy functionality...")
    from spacy.tokens import Doc
    print("✅ spaCy.tokens.Doc imported successfully")
    
    print("\n" + "=" * 80)
    print("🎉 All dependencies loaded successfully!")
    print("=" * 80)
    print("\nYou can now proceed to test the model.")
    
except ImportError as e:
    print(f"\n❌ Import error: {e}")
    print("\n⚠️  Please check the installed versions above.")
except Exception as e:
    print(f"\n❌ Unexpected error: {e}")
    print(f"\n⚠️  Error type: {type(e).__name__}")
    print("\nThis suggests spaCy/pydantic are still incompatible.")
    print("See the versions listed above - they should be:")
    print("   spacy: 3.7.4")
    print("   pydantic: 2.6.0 or 2.6.x")

🔍 Verifying installation...

📦 Checking installed versions:
   fastapi                                  0.109.0
   opentelemetry-instrumentation-fastapi    0.59b0
   pydantic                                 2.6.0
   pydantic_core                            2.16.1
   pydantic-settings                        2.1.0
   spacy                                    3.7.4
   spacy-alignments                         0.9.2
   spacy-legacy                             3.0.12
   spacy-loggers                            1.0.5
   spacy-transformers                       1.3.5
   thinc                                    8.2.5

🧪 Testing imports...

❌ Unexpected error: ForwardRef._evaluate() missing 1 required keyword-only argument: 'recursive_guard'

⚠️  Error type: TypeError

This suggests spaCy/pydantic are still incompatible.
See the versions listed above - they should be:
   spacy: 3.7.4
   pydantic: 2.6.0 or 2.6.x


In [5]:
# 安装Python 3.10
!sudo apt-get update -qq
!sudo apt-get install -y python3.10 python3.10-dev python3.10-distutils
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
!python3.10 --version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.10-distutils'
python3-distutils is already the newest version (3.10.8-1~22.04).
python3-distutils set to manually installed.
python3.10 is already the newest version (3.10.12-1~22.04.11).
python3.10 set to manually installed.
The following NEW packages will be installed:
  python3.10-dev
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 508 kB of archives.
After this operation, 523 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3.10-dev amd64 3.10.12-1~22.04.11 [508 kB]
Fetched 508 kB in 1s (638 kB/s)         
debconf: unable to initialize frontend: Dialog
debc

In [6]:
!python nlp/scripts/test_trained_model.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/Helixgraph/nlp/scripts/test_trained_model.py", line 8, in <module>
    import spacy
  File "/usr/local/lib/python3.12/dist-packages/spacy/__init__.py", line 13, in <module>
    from . import pipeline  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/spacy/pipeline/__init__.py", line 1, in <module>
    from .attributeruler import AttributeRuler
  File "/usr/local/lib/python3.12/dist-packages/spacy/pipeline/attributeruler.py", line 8, in <module>
    from ..language import Language
  File "/usr/local/lib/python3.12/dist-packages/spacy/language.py", line 43, in <module>
    from .pipe_analysis import analyze_pipes, print_pipe_analysis, validate_attrs
  File "/usr/local/lib/python3.12/dist-packages/spacy/pipe_analysis.py", line 6, in <module>
    from .tokens import Doc, Span, Token
  File "/usr/local/lib/python3.12/dist-packages/spacy/tokens/__init__.py", line 1, in <module>
    from .

## 🔗 Step 3: Entity Linking开发与测试

代码文件: `nlp/entity_linking.py`

In [2]:
# 测试Entity Linking
!python nlp/scripts/test_entity_linking.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/Helixgraph/nlp/scripts/test_entity_linking.py", line 16, in <module>
    import spacy
  File "/usr/local/lib/python3.12/dist-packages/spacy/__init__.py", line 13, in <module>
    from . import pipeline  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/spacy/pipeline/__init__.py", line 1, in <module>
    from .attributeruler import AttributeRuler
  File "/usr/local/lib/python3.12/dist-packages/spacy/pipeline/attributeruler.py", line 8, in <module>
    from ..language import Language
  File "/usr/local/lib/python3.12/dist-packages/spacy/language.py", line 43, in <module>
    from .pipe_analysis import analyze_pipes, print_pipe_analysis, validate_attrs
  File "/usr/local/lib/python3.12/dist-packages/spacy/pipe_analysis.py", line 6, in <module>
    from .tokens import Doc, Span, Token
  File "/usr/local/lib/python3.12/dist-packages/spacy/tokens/__init__.py", line 1, in <module>
    from

## 🚀 Step 4: FastAPI应用开发

代码文件: `api/main.py`

In [ ]:
# 启动FastAPI服务器 (在后台运行)
import nest_asyncio
nest_asyncio.apply()

# 使用nohup在后台运行
!nohup uvicorn api.main:app --host 0.0.0.0 --port 8000 > api.log 2>&1 &

print("🚀 FastAPI server starting...")
print("Check logs: !tail -f api.log")

In [ ]:
# 查看API日志
!tail -20 api.log

In [ ]:
# 测试API健康检查
import requests

response = requests.get("http://localhost:8000/health")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

## 🌐 Step 5: 设置Cloudflare Tunnel (可选)

如果需要外部访问

In [ ]:
# 使用ngrok创建tunnel
from pyngrok import ngrok

# 创建tunnel到FastAPI
public_url = ngrok.connect(8000)
print(f"\n🌐 Public URL: {public_url}")
print(f"\nTest with: curl {public_url}/health")

## 🧪 Step 6: 集成测试

运行完整的集成测试套件

In [ ]:
# 运行pytest测试
!pytest tests/ -v

In [ ]:
# 运行特定测试
!pytest tests/test_entity_linking.py -v

## 📊 Step 7: 手动测试API Endpoints

In [ ]:
# 测试NER提取
import requests

response = requests.post(
    "http://localhost:8000/api/extract-entities",
    json={"text": "Tech Solutions Ltd submitted PO-2024-001 for the Spring Launch campaign."}
)

print(f"Status: {response.status_code}")
print(f"Entities: {response.json()}")

In [ ]:
# 测试Entity Linking
response = requests.post(
    "http://localhost:8000/api/link-entities",
    json={
        "text": "Acme Corp submitted invoice INV-123456",
        "entities": [
            {"text": "Acme Corp", "label": "SUPPLIER", "start": 0, "end": 9},
            {"text": "INV-123456", "label": "INVOICE", "start": 29, "end": 39}
        ]
    }
)

print(f"Linked entities: {response.json()}")

## 🛑 Step 8: 停止服务器

In [ ]:
# 停止FastAPI服务器
!pkill -f "uvicorn api.main:app"
print("✅ Server stopped")

---

## 📝 开发笔记

### 文件结构
```
nlp/
├── entity_linking.py          # Entity linking实现
├── entity_extraction.py       # RAG helper
└── scripts/
    ├── test_trained_model.py  # 模型测试
    └── test_entity_linking.py # Entity linking测试

api/
├── main.py                    # FastAPI应用
├── config.py                  # 配置
├── database.py                # Neo4j连接
└── endpoints/
    ├── ner.py                 # NER endpoints
    └── fixed_queries.py       # 4个固定查询

tests/
├── test_entity_linking.py
├── test_api.py
└── test_integration.py
```

### 下一步
- [ ] 完成Entity Linking实现
- [ ] 创建FastAPI基础框架
- [ ] 实现4个固定查询
- [ ] 与Sun (Neo4j) 和 Mert (RAG) 集成